In [7]:
import pandas as pd
import os
from collections import defaultdict
from numpy import nan
import pickle,os,csv,math
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from optparse import OptionParser

In [8]:
path = '../df_zebrafish.csv'
df = pd.read_csv(path)
df

,name,effect,rs28399504,SSTR1,AR,"CYP2D6*1, CYP2D6*10, CYP2D6*2, CYP2D6*3, CYP2D6*4, CYP2D6*41, CYP2D6*5, CYP2D6*6",rs2900478,TAS2R50,CYP2A6,rs1126647,...,UGT2B15,SULT1A2,rs9282861,SULT1A1,PTGES3,GABRA6,GRIA2,CHRNA7,GRIK2,GRIA4
0,DB00396,0.759465,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,DB00292,0.697795,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DB00602,0.620809,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,DB01093,0.401215,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DB11148,0.379060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,DB00903,-0.027501,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323,DB00316,-0.029771,0,0,0,1,0,0,1,0,...,1,1,1,1,1,0,0,0,0,0
324,DB00270,-0.032226,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
325,DB00794,-0.032435,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1


In [9]:
# save drugnames and drug banks ids but remove them for the regression
dbids = df['name'].to_list()
df = df.drop(['name'],axis=1)
#df = df.drop(['Unnamed: 0',],axis=1)

y = df.loc[:,'effect']
X = df.drop(['effect'],axis=1)
gnames = list(X.columns)

split_row = math.ceil(X.shape[0]*0.2) # save 20% of rows for testing
X_test = X.loc[:split_row,:]
y_test = y.loc[:split_row,]
X_train = X.loc[split_row:,:]
y_train = y.loc[split_row:,]

alpha = 1.0
model = Ridge(alpha=alpha, random_state=0)
model.fit(X,y)
predicted_classes = model.predict(X)

print('Mean squared error: %.2f'% mean_squared_error(y, predicted_classes))
print('Coefficient of determination: %.2f'% r2_score(y, predicted_classes))

Mean squared error: 0.00
Coefficient of determination: 0.63


In [10]:
# Access the coefficients
coefficients = model.coef_
coefficients

array([ 0.00093639, -0.00079815,  0.02200758, ..., -0.03223951,
       -0.03223951, -0.03223951])

In [12]:
reg_coef = list(zip(gnames, coefficients))
sort_rc = sorted(reg_coef,key = lambda x:x[1],reverse=True)
out_df = pd.DataFrame({'Gene Names':gnames,'Regression Coefficients':coefficients})
out_df = out_df.sort_values(by='Regression Coefficients', ascending=False)
out_df.to_excel('zebrafish_ridgeregcoeffs.xlsx',index=False)

In [6]:
outf = "drugScreenRidgeRegression2.xlsx"
writer = pd.ExcelWriter(outf,engine='xlsxwriter')
reg_coef = list(zip(gnames, model.coef_))
sort_rc = sorted(reg_coef,key = lambda x:x[1],reverse=True)
out_df = pd.DataFrame({'Gene Names':gnames,'Regression Coefficients':model.coef_[0]})
out_df = out_df.sort_values(by='Regression Coefficients', ascending=False)
out_df.to_excel(writer,sheet_name=outf,index=False)
writer.save()